In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#export
import torch
import torch.nn.functional as F
from torch import nn
from enum import Enum, auto

In [5]:
class ProbClassifier(nn.Module):
    def __init__(self,kernel_size = 3 , ):
        self.kernel_size = kernel_size
        
        
        K = self.kernel_size
        self.filter_shape = (K // 2 + 1, K, K) # CHW
        
    def create_first_mask(self):
            """create 5d mask that includes all pixel's in strictly before"""
            K = self.kernel_size
            # mask is DHW
            mask = torch.ones(self.filter_shape, dtype=torch.float32, requires_grad=False)
            # zero out D=1,
            # - everything to the right of the central pixel, including the central pixel
            mask[-1, K // 2, K // 2:] = 0
            # - all rows below the central row
            mask[-1, K // 2 + 1:, :] = 0

            mask.unsqueeze_(-1).unsqueeze_(-1)  # Make into DHWio, for broadcasting with 3D filters
            return nn.Parameter(mask)
        
    def create_other_mask(self):
        """create 5d mask that includes all pixel's in before and current pixel"""
        mask = self.create_first_mask()
        mask[-1, K // 2, K // 2 + 1] = 1
        return mask
    
    @staticmethod
    def pad_for_probclass3d(x, context_size, pad_value=0, learn_pad_var=False):
    """
    :param x: NCHW tensor 
    """
        pad = context_size // 2
        assert pad >= 1
        if learn_pad_var:
            if not isinstance(pad_value, tf.Variable):
                print('Warn: Expected tf.Variable for padding, got {}'.format(pad_value))
            return pc_pad_grad(x, pad, pad_value)

        pads = [[0, 0],  # don't pad batch dimension
                [pad, 0],  # don't pad depth_future, it's not seen by any filter
                [pad, pad],
                [pad, pad]]
        assert len(pads) == _get_ndims(x), '{} != {}'.format(len(pads), x.shape)

        pad_fn = tf.pad if input_is_tf else get_np_pad_fn()
        return pad_fn(x, pads, constant_values=pad_value)

In [ ]:
 def bitcost(self, q, target_symbols, is_training, pad_value=0):
        """
        Pads q, creates PC network, calculates cross entropy between output of PC network and target_symbols
        :param q: NCHW
        :param target_symbols:
        :param is_training:
        :return: bitcost per symbol: NCHW
        """
        tf_helpers.assert_ndims(q, 4)

        with self._building_ctx(self.reuse):
            if self.first_mask is None:
                self.first_mask = self.create_first_mask()  # DHWio
                self.other_mask = self.create_other_mask()  # DHWio

            self.reuse = True

            targets_one_hot = tf.one_hot(target_symbols, depth=self.L, axis=-1, name='target_symbols')

            q_pad = pad_for_probclass3d(
                    q, context_size=self.get_context_size(self.config),
                    pad_value=pad_value, learn_pad_var=False)
            with tf.variable_scope('logits'):
                # make it into NCHWT, where T is the channel dim of the conv3d
                q_pad = tf.expand_dims(q_pad, -1, name='NCHWT')
                logits = self._logits(q_pad, is_training)

            if self.config.regularization_factor is not None:
                print('Creating PC regularization...')
                weights = _get_all_conv3d_weights_in_scope(self._PROBCLASS_SCOPE)
                assert len(weights) > 0
                reg = self.config.regularization_factor * tf.add_n(list(map(tf.nn.l2_loss, weights)))
                tf.losses.add_loss(reg, tf.GraphKeys.REGULARIZATION_LOSSES)

            if targets_one_hot.shape.is_fully_defined() and logits.shape.is_fully_defined():
                tf_helpers.assert_equal_shape(targets_one_hot, logits)

            with tf.name_scope('bitcost'):
                # softmax_cross_entropy_with_logits is basis e, change base to 2
                log_base_change_factor = tf.constant(np.log2(np.e), dtype=tf.float32)
                bc = tf.nn.softmax_cross_entropy_with_logits(
                    logits=logits, labels=targets_one_hot) * log_base_change_factor  # NCHW

            return bc # bit cost